In [1]:
from rasa_nlu.training_data import load_data
from rasa_nlu.model import Trainer, Interpreter
from rasa_nlu import config
import json

def test_train_and_persist():
    training_data = load_data("data/nlu_data.md")
    trainer = Trainer(config.load("nlu_config.yml"))
    interpreter = trainer.train(training_data)
    test_interpreter_dir = trainer.persist("./tests/models", project_name="nlu")
    parsing = interpreter.parse('hello')

    assert parsing['intent']['name'] == 'greet'
    assert test_interpreter_dir

In [3]:
training_data = load_data("data/train.md")
trainer = Trainer(config.load("nlu_config.yml"))
trainer

In [2]:
interpreter = Interpreter.load("./models/current/default/nlu/")
def parse(message):
    result = interpreter.parse(message) 
    print(json.dumps(result, indent=2))
    return result

INFO:tensorflow:Restoring parameters from ./models/current/default/nlu/intent_classifier_tensorflow_embedding.ckpt


In [8]:
x = parse('follow in to the left rack')
trainer.persist()

{
  "intent": {
    "name": "move",
    "confidence": 0.7862564921379089
  },
  "entities": [
    {
      "start": 17,
      "end": 26,
      "value": "left rack",
      "entity": "location",
      "confidence": 0.9978338381105102,
      "extractor": "ner_crf"
    }
  ],
  "intent_ranking": [
    {
      "name": "move",
      "confidence": 0.7862564921379089
    },
    {
      "name": "follow",
      "confidence": 0.5004448890686035
    },
    {
      "name": "bye",
      "confidence": 0.0
    },
    {
      "name": "affirmative",
      "confidence": 0.0
    },
    {
      "name": "stop",
      "confidence": 0.0
    },
    {
      "name": "start",
      "confidence": 0.0
    },
    {
      "name": "greet",
      "confidence": 0.0
    },
    {
      "name": "negative",
      "confidence": 0.0
    }
  ],
  "text": "follow in to the left rack"
}


NameError: name 'trainer' is not defined

In [1]:
import rasa_core
from rasa_core.agent import Agent
import time

messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
agent = Agent.load('models/dialogue')

def chatlogs_html(messages):
    messages_html = "".join(["<p>{}</p>".format(m) for m in messages])
    chatbot_html = """<div class="chat-window" {}</div>""".format(messages_html)
    return chatbot_html


while True:
    clear_output()
    display(HTML(chatlogs_html(messages)))
    time.sleep(0.3)
    a = input()
    messages.append(a)
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for r in responses:
        messages.append(r.get("text"))

KeyboardInterrupt: 

In [82]:
from rasa_core.train import train_dialogue_model
from rasa_core import config
from rasa_core.agent import Agent
from rasa_core.policies.keras_policy import KerasPolicy
import rasa_core
policies = config.load('policy_config.yml')
agent = Agent(domain = 'domain.yml', policies = policies)
data = agent.load_data('data/stories.md')
agent.train(data)
agent.persist('models/dialogue')

Processed Story Blocks: 100%|██████████| 1/1 [00:00<00:00, 1155.77it/s, # trackers=1]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 35)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8704      
_________________________________________________________________
dense (Dense)                (None, 10)                330       
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 9,034
Trainable params: 9,034
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
11/11 [==============================] - 1s 56ms/step - loss: 2.2993 - acc: 0.0909
Epoch 2/500
11/11 [==============================] - 0s 753us/step - loss: 2.2823 - acc: 0.0909
Epoch 3/500
11/11 [==============================] - 0s 875us/step - loss: 2.2478

Processed actions: 11it [00:00, 2136.38it/s, # examples=11]


In [1]:
from rasa_core.agent import Agent
from rasa_core.interpreter import RasaNLUInterpreter
from rasa_core.utils import EndpointConfig

interpreter = RasaNLUInterpreter('models/current/default/nlu')
messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
agent = Agent.load('models/dialogue', interpreter=interpreter, action_endpoint=EndpointConfig(url="http://0.0.0.0:5055/webhook"))

INFO:tensorflow:Restoring parameters from models/current/default/nlu/intent_classifier_tensorflow_embedding.ckpt


In [2]:
agent.handle_message('move to the kitchen')

[{'recipient_id': 'default', 'text': 'Hello there, how can a I help you?'}]

In [10]:
agent.interpreter.parse('Hello there, how can a I help you?')

{'intent': {'name': 'greet', 'confidence': 0.9721769094467163},
 'entities': [],
 'intent_ranking': [{'name': 'greet', 'confidence': 0.9721769094467163},
  {'name': 'stop', 'confidence': 0.0},
  {'name': 'follow', 'confidence': 0.0},
  {'name': 'negative', 'confidence': 0.0},
  {'name': 'start', 'confidence': 0.0},
  {'name': 'affirmative', 'confidence': 0.0},
  {'name': 'bye', 'confidence': 0.0},
  {'name': 'move', 'confidence': 0.0}],
 'text': 'Hello there, how can a I help you?'}

In [70]:
python -m rasa_core.train \
  interactive -o models/dialogue \
  -d domain.yml -c policy_config.yml \
  -s data/stories.md \
  --nlu models/current/default/nlu \
  --endpoints endpoints.yml

{'scores': [{'action': 'action_listen', 'score': 0.07884559599995151},
  {'action': 'action_restart', 'score': 0.00032237497352218696},
  {'action': 'action_default_fallback', 'score': 0.00023362266995165793},
  {'action': 'action_deactivate_form', 'score': 0.0002530063153575476},
  {'action': 'action_revert_fallback_events', 'score': 0.00027700521698785887},
  {'action': 'action_default_ask_affirmation', 'score': 0.000289754906327366},
  {'action': 'action_default_ask_rephrase', 'score': 0.00026343491606356566},
  {'action': 'utter_greet', 'score': 0.8846614754484059},
  {'action': 'action_planner', 'score': 0.03485372955343245}],
 'policy': 'policy_0_KerasPolicy',
 'confidence': 0.8846614754484059,
 'tracker': {'sender_id': 1,
  'slots': {'anything': None,
   'direction': None,
   'gender': None,
   'gesture': None,
   'location': None,
   'name': None,
   'object': None,
   'pronoun': None},
  'latest_message': {'intent': {}, 'entities': [], 'text': None},
  'latest_event_time': 155